In [1]:
import numpy as np
from asdf import AsdfFile
from astropy.io import fits
from astropy import wcs as astwcs
from astropy import table

from gwcs import wcs
from jwst import datamodels
from jwst.assign_wcs import nirspec
from jwst.transforms import models

In [2]:
#t = table.Table.read('/Users/dencheva/Downloads/fixed_slits_functional_ESA.txt', format='ascii')
t4 = table.Table.read('/home/dencheva/data/astropy/test-nrs/nrs_testing/test_model/fixed_slits_functional_ESA_v4_20180618.txt', format='ascii')
#t4.columns

In [3]:
#Create reference file dict
def create_reference_files(datamodel):
    """
    Create a dict {reftype: reference_file}.
    """
    refs = {}
    step = assign_wcs_step.AssignWcsStep()
    for reftype in assign_wcs_step.AssignWcsStep.reference_file_types:
        refs[reftype] = step.get_reference_file(datamodel, reftype)
    return refs

# These are the CV3 files
refs = {'camera': '/grp/crds/cache/references/jwst/jwst_nirspec_camera_0004.asdf',
        'collimator': '/grp/crds/cache/references/jwst/jwst_nirspec_collimator_0004.asdf',
        'disperser': '/grp/crds/cache/references/jwst/jwst_nirspec_disperser_0035.asdf',
        'distortion': 'N/A',
        'filteroffset': 'N/A',
        'fore': '/grp/crds/cache/references/jwst/jwst_nirspec_fore_0022.asdf',
        'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf',
        'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf',
        'ifupost': 'ifupost.asdf',
        'ifuslicer': '/grp/crds/cache/references/jwst/jwst_nirspec_ifuslicer_0003.asdf',
        'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf',
        'ote': 'ote.asdf',
        'regions': 'N/A',
        'specwcs': 'N/A',
        'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf'}


In [4]:
wcs_kw = {'wcsaxes': 2, 'ra_ref': 165, 'dec_ref': 54,
          'v2_ref': -8.3942412, 'v3_ref': -5.3123744, 'roll_ref': 37,
          'crpix1': 1024, 'crpix2': 1024,
          'cdelt1': .08, 'cdelt2': .08,
          'ctype1': 'RA---TAN', 'ctype2': 'DEC--TAN',
          'pc1_1': 1, 'pc1_2': 0, 'pc2_1': 0, 'pc2_2': 1
          }

slit_fields_num = ["shutter_id", "xcen", "ycen",
                   "ymin", "ymax", "quadrant", "source_id",
                   "stellarity", "source_xpos", "source_ypos"]


slit_fields_str = ["name", "shutter_state", "source_name", "source_alias"]


In [5]:
def create_hdul(detector='NRS1'):
    """
    Create a fits HDUList instance.
    """
    hdul = fits.HDUList()
    phdu = fits.PrimaryHDU()
    phdu.header['instrume'] = 'NIRSPEC'
    phdu.header['detector'] = detector
    phdu.header['time-obs'] = '8:59:37'
    phdu.header['date-obs'] = '2016-09-05'

    scihdu = fits.ImageHDU()
    scihdu.header['EXTNAME'] = "SCI"
    for item in wcs_kw.items():
        scihdu.header[item[0]] = item[1]
    hdul.append(phdu)
    hdul.append(scihdu)
    return hdul

In [6]:
def create_nirspec_fs_file(grating, filter, lamp="N/A", detector='NRS1'):
    image = create_hdul(detector)
    image[0].header['exp_type'] = 'NRS_FIXEDSLIT'
    image[0].header['filter'] = filter
    image[0].header['grating'] = grating
    image[0].header['lamp'] = lamp
    image[1].header['crval3'] = 0
    image[1].header['wcsaxes'] = 3
    image[1].header['ctype3'] = 'WAVE'
    image[0].header['GWA_XTIL'] = 0.3316612243652344
    image[0].header['GWA_YTIL'] = 0.1260581910610199
    image[0].header['SUBARRAY'] = "FULL"
    return image



In [7]:
hdul = create_nirspec_fs_file(grating='G395H', filter='F290LP',  detector='NRS1')
im = datamodels.ImageModel(hdul)

pipeline = nirspec.create_pipeline(im, refs)
w = wcs.WCS(pipeline)
im.meta.wcs = w


2018-06-19 13:26:07,229 - stpipe - INFO - gwa_ytilt is 0.1260581910610199 deg
2018-06-19 13:26:07,230 - stpipe - INFO - gwa_xtilt is 0.3316612243652344 deg
2018-06-19 13:26:07,231 - stpipe - INFO - theta_y correction: -0.009545474118238594 deg
2018-06-19 13:26:07,232 - stpipe - INFO - theta_x correction: 0.0 deg
2018-06-19 13:26:07,958 - stpipe - INFO - Removing slit S200B1 from the list of open slits because the WCS bounding_box is completely outside the detector.
2018-06-19 13:26:07,959 - stpipe - INFO - Slits projected on detector NRS1: ['S200A1', 'S200A2', 'S400A1', 'S1600A1']
2018-06-19 13:26:07,960 - stpipe - INFO - Computing WCS for 4 open slitlets
2018-06-19 13:26:07,979 - stpipe - INFO - gwa_ytilt is 0.1260581910610199 deg
2018-06-19 13:26:07,980 - stpipe - INFO - gwa_xtilt is 0.3316612243652344 deg
2018-06-19 13:26:07,981 - stpipe - INFO - theta_y correction: -0.009545474118238594 deg
2018-06-19 13:26:07,982 - stpipe - INFO - theta_x correction: 0.0 deg
2018-06-19 13:26:07,99

In [8]:
# Setup the test
slitx = [0] * 5
slity = [-.5, -.25, 0, .25, .5]
wave_range = [2.87e-06, 5.27e-06]
lam = np.array([2.9, 3.39, 3.88, 4.37, 5]) * 10**-6

# Use slit S200A1
slit_wcs = nirspec.nrs_wcs_set_input(im, 'S200A1')

In [9]:
# Slit to MSA absolute
slit2msa = slit_wcs.get_transform('slit_frame', 'msa_frame')
msax, msay, _= slit2msa(slitx, slity, lam)
print('slitx: ', slitx )
print('slity: ', slity)
print('msax: ', msax)
print('msay: ', msay)
print('diff slitx', slitx - t4['xslitpos'])
print('diff slity', slity - t4['yslitpos'])
print('diff x_msa', msax - t4['xmsapos'])
print('diff y_msa', msay - t4['ymaspos'])

slitx:  [0, 0, 0, 0, 0]
slity:  [-0.5, -0.25, 0, 0.25, 0.5]
msax:  [ 0.02697243  0.02697243  0.02697243  0.02697243  0.02697243]
msay:  [-0.00335229 -0.00303449 -0.0027167  -0.00239891 -0.00208112]
diff slitx xslitpos
--------
       0
       0
       0
       0
       0
diff slity yslitpos
--------
     0.0
     0.0
     0.0
     0.0
     0.0
diff x_msa xmsapos
-------
    0.0
    0.0
    0.0
    0.0
    0.0
diff y_msa ymaspos
-------
    0.0
    0.0
    0.0
    0.0
    0.0


In [10]:
# Coordinates at Collimator exit
# Applies the Collimator forward transform to MSa absolute coordinates
col = datamodels.open(refs['collimator'])
colx, coly = col.model.inverse(msax, msay)
print('x_collimator_exit', colx)
print('y_collimator_exit', coly)
col.close()
print('diff x_collimator_exit', colx - t4['xcoll'])
print('diff y_collimator_exit', coly - t4['ycoll'])

x_collimator_exit [ 0.0403573   0.04035758  0.04035787  0.04035816  0.04035847]
y_collimator_exit [-0.3004114  -0.29988724 -0.29936322 -0.29883931 -0.29831554]
diff x_collimator_exit       xcoll       
------------------
 6.93889390391e-18
               0.0
               0.0
 6.93889390391e-18
-6.93889390391e-18
diff y_collimator_exit       ycoll       
------------------
 1.11022302463e-16
               0.0
-5.55111512313e-17
 5.55111512313e-17
-5.55111512313e-17


In [11]:
# After applying direcitonal cosines
from jwst.transforms import models as trmodels

dircos = trmodels.Unitless2DirCos()
xcolDircosi, ycolDircosi, z = dircos(colx, coly)
print('xcolDircosi ', xcolDircosi)
print('ycolDircosi ', ycolDircosi)
print('z', z)
print('diff xcolDircosi', xcolDircosi - t4['xcolDirCosi'])
print('diff ycolDircosi', ycolDircosi - t4['ycolDirCosi'])

xcolDircosi  [ 0.03862206  0.0386279   0.03863373  0.03863956  0.0386454 ]
ycolDircosi  [-0.28749467 -0.28703441 -0.28657407 -0.28611363 -0.28565311]
z [ 0.95700321  0.95714112  0.95727882  0.9574163   0.95755357]
diff xcolDircosi    xcolDirCosi    
------------------
 6.93889390391e-18
               0.0
               0.0
 6.93889390391e-18
-1.38777878078e-17
diff ycolDircosi    ycolDirCosi    
------------------
 1.11022302463e-16
               0.0
-5.55111512313e-17
 5.55111512313e-17
               0.0


In [12]:
# MSA to GWA entrance
# This runs the Collimator forward, Unitless to Directional cosine, and 3D Rotation
# It uses the corrected GWA tilt value
disp = datamodels.DisperserModel(refs['disperser'])

In [13]:
print(disp.meta.instrument.grating)
print('theta_y: ', disp.theta_y)
print('theta_y: ', disp.theta_x)


G395H
theta_y:  -0.03089246914388889
theta_y:  0.023015465167583335


In [14]:
disperser = nirspec.correct_tilt(disp, im.meta.instrument.gwa_xtilt, im.meta.instrument.gwa_ytilt)

print('theta_y corrected: ', disperser.theta_y)
print('theta_y: ', disperser.theta_x)
collimator2gwa = nirspec.collimator_to_gwa(refs, disperser)
x_gwa_in, y_gwa_in, z_gwa_in = collimator2gwa(msax, msay)
disp.close()
print('x_gwa_entrance:' , x_gwa_in)
print('y_gwa_entrance:' , y_gwa_in)
print('z_gwa_entrance:' , z_gwa_in)
print(-0.009545474118238594*3600)

print('diff x_gwa_entrance', x_gwa_in - t4['xdispIn'])
print('diff y_gwa_entrance', y_gwa_in - t4['ydispIn'])

2018-06-19 13:26:10,506 - stpipe - INFO - gwa_ytilt is 0.1260581910610199 deg
2018-06-19 13:26:10,507 - stpipe - INFO - gwa_xtilt is 0.3316612243652344 deg
2018-06-19 13:26:10,508 - stpipe - INFO - theta_y correction: -0.009545474118238594 deg
2018-06-19 13:26:10,509 - stpipe - INFO - theta_x correction: 0.0 deg


theta_y corrected:  -0.04043794326212748
theta_y:  0.023015465167583335
x_gwa_entrance: [ 0.18738332  0.18740684  0.18743033  0.18745378  0.1874772 ]
y_gwa_entrance: [-0.28680741 -0.28634706 -0.28588663 -0.2854261  -0.28496549]
z_gwa_entrance: [ 0.93948337  0.93961909  0.9397546   0.9398899   0.94002498]
-34.36370682565894
diff x_gwa_entrance      xdispIn      
------------------
 2.77555756156e-17
               0.0
               0.0
               0.0
-2.77555756156e-17
diff y_gwa_entrance      ydispIn      
------------------
 1.11022302463e-16
               0.0
-5.55111512313e-17
 5.55111512313e-17
               0.0


In [15]:
# Slit to GWA out
slit2gwa = slit_wcs.get_transform('slit_frame', 'gwa')
x_gwa_out, y_gwa_out, z_gwa_out = slit2gwa(slitx, slity, lam)
print('x_gwa_exit:' , x_gwa_out)
print('y_gwa_exit:' , y_gwa_out)
print('z_gwa_exit:' , z_gwa_out)

print('diff x_gwa_exit', x_gwa_out - t4['xdispLaw'])
print('diff y_gwa_exit', y_gwa_out - t4['ydispLaw'])

x_gwa_exit: [ 0.07190191  0.11568865  0.15947543  0.20326224  0.2595663 ]
y_gwa_exit: [ 0.28680741  0.28634706  0.28588663  0.2854261   0.28496549]
z_gwa_exit: [ 0.95528615  0.95111592  0.94490022  0.93659831  0.92272423]
diff x_gwa_exit      xdispLaw     
------------------
-2.77555756156e-17
               0.0
               0.0
               0.0
 5.55111512313e-17
diff y_gwa_exit      ydispLaw     
------------------
-1.11022302463e-16
               0.0
 5.55111512313e-17
-5.55111512313e-17
               0.0


In [16]:
# CAMERA entrance (assuming direction is from sky to detector)
angles = [disperser['theta_x'], disperser['theta_y'],
          disperser['theta_z'], disperser['tilt_y']]
rotation = models.Rotation3DToGWA(angles, axes_order="xyzy", name='rotation')
dircos2unitless = models.DirCos2Unitless()

gwa2cam = rotation.inverse | dircos2unitless
x_camera_entrance, y_camera_entrance = gwa2cam(x_gwa_out, y_gwa_out, z_gwa_out)
print('x_camera_entrance:' , x_camera_entrance)
print('y_camera_entrance:' , y_camera_entrance)

print('diff x_camera_enrtance', x_camera_entrance - t4['xcamCosi'])
print('diff y_camera_entrance', y_camera_entrance - t4['ycamCosi'])

x_camera_entrance: [-0.07707064 -0.03101948  0.01514077  0.06171155  0.12272885]
y_camera_entrance: [ 0.30046859  0.29882465  0.29781475  0.29744381  0.29808709]
diff x_camera_enrtance      xcamCosi     
------------------
-5.55111512313e-17
               0.0
               0.0
-2.77555756156e-17
 6.93889390391e-17
diff y_camera_entrance      ycamCosi     
------------------
-1.11022302463e-16
               0.0
 1.11022302463e-16
-5.55111512313e-17
               0.0


In [18]:
# at FPA
camera = datamodels.CameraModel(refs['camera'])
x_fpa, y_fpa = camera.model.inverse(x_camera_entrance, y_camera_entrance)
camera.close()
print('x_fpa: ', x_fpa )
print('y_fpa: ' , y_fpa)

print('diff x_fpa', x_fpa - t4['xfpapos'])
print('diff y_fpa', y_fpa - t4['yfpapos'])

x_fpa:  [-0.02239022 -0.00937323  0.00372075  0.01691372  0.03406922]
y_fpa:  [ 0.00142652  0.00119317  0.00103662  0.0009563   0.00101618]
diff x_fpa      xfpapos      
------------------
-1.73472347598e-17
 1.73472347598e-18
               0.0
-1.04083408559e-17
 6.93889390391e-18
diff y_fpa      yfpapos      
------------------
-2.99239799606e-17
 4.33680868994e-19
 2.97071395261e-17
-1.46367293286e-17
 4.33680868994e-19


In [19]:
# at SCA These are 0-based , the IDT results are 1-based
slit2sca = slit_wcs.get_transform('slit_frame', 'sca')
x_sca_nrs1, y_sca_nrs1 = slit2sca(slitx, slity, lam)
#fpa = datamodels.open(refs['fpa'])
#print('fpax, fpay', fpa.nrs1_model.inverse(x_fpa, y_fpa))
# At NRS2
fpa = datamodels.FPAModel(refs['fpa'])
fpa.nrs1_model
x_sca_nrs2, y_sca_nrs2 = fpa.nrs2_model.inverse(x_fpa, y_fpa)
fpa.close()
print('x_sca1: ', x_sca_nrs1)
print('y_sca1: ' , y_sca_nrs1)
print('x_sca2: ', x_sca_nrs2 )
print('y_sca2: ' , y_sca_nrs2)

print('diff x_sca1', x_sca_nrs1[:2] - t4['i'][:2])
print('diff y_sca1', y_sca_nrs1[:2] - t4['j'][:2])
print('\n')
print('diff x_sca2', x_sca_nrs2[2:4] - t4['i'][2:4])
print('diff y_sca2', y_sca_nrs2[2:4] - t4['j'][2:4])

x_sca1:  [  876.70094739  1599.86683035  2327.31023955  3060.25306449  4013.33665474]
y_sca1:  [ 1102.75121505  1089.78694793  1081.08997251  1076.6275921   1079.95467952]
x_sca2:  [ 3364.51728613  2641.35159688  1913.90831766  1180.96555945   227.88191961]
y_sca2:  [ 944.16217632  957.13724461  965.84508511  970.31841273  967.00556052]
diff x_sca1  i  
----
-1.0
-1.0
diff y_sca1  j  
----
-1.0
-1.0


diff x_sca2        i       
---------------
           -1.0
-0.999999999999
diff y_sca2        j       
---------------
           -1.0
-0.999999999999


In [20]:
# at oteip
slit2oteip = slit_wcs.get_transform('slit_frame', 'oteip')
x_oteip, y_oteip, _ = slit2oteip(slitx, slity, lam)
print('x_oteip: ', x_oteip)
print('y_oteip: ' , y_oteip)

print('diff x_oteip', x_oteip - t4['xOTEIP'])
print('diff y_oteip', y_oteip - t4['yOTEIP'])

x_oteip:  [ 0.02964448  0.02999172  0.03033894  0.03068614  0.03103378]
y_oteip:  [-0.03278024 -0.03238316 -0.03198612 -0.03158912 -0.0311919 ]
diff x_oteip       xOTEIP      
------------------
               0.0
-3.46944695195e-18
 3.46944695195e-18
-1.04083408559e-17
               0.0
diff y_oteip       yOTEIP      
------------------
               0.0
 6.93889390391e-18
-6.93889390391e-18
 6.93889390391e-18
 6.93889390391e-18


In [21]:
# at v2, v3 [in arcsec]
slit2v23 = slit_wcs.get_transform('slit_frame', 'v2v3')
v2, v3, _ = slit2v23(slitx, slity, lam)
v2 /= 3600
v3 /= 3600
print('v2: ', v2)
print('v3: ' , v3)

print('diff V2', v2 - t4['xV2V3'])
print('diff V3', v3 - t4['yV2V3'])

v2:  [ 0.09254187  0.09238998  0.09223809  0.09208622  0.09193416]
v3:  [-0.13344017 -0.13326734 -0.13309453 -0.13292174 -0.13274886]
diff V2       xV2V3      
-----------------
              0.0
1.38777878078e-17
1.38777878078e-17
1.38777878078e-17
              0.0
diff V3       yV2V3       
------------------
               0.0
               0.0
-2.77555756156e-17
               0.0
               0.0


In [22]:
# Save results to an asdf file

fa = AsdfFile()
fa.tree['slitx'] = list(slitx)
fa.tree['slity'] = list(slity)
fa.tree['lam'] = list(lam)
fa.tree['msax'] = list(msax)
fa.tree['msay'] = list(msay)
fa.tree['x_collimator_exit'] = list(colx)
fa.tree['y_collimator_exit'] = list(coly)
fa.tree['x_gwa_entrance'] = list(x_gwa_in)
fa.tree['y_gwa_entrance'] = list(y_gwa_in)
fa.tree['z_gwa_entrance'] = list(z_gwa_in)
fa.tree['x_gwa_exit'] = list(x_gwa_out)
fa.tree['y_gwa_exit'] = list(y_gwa_out)
fa.tree['z_gwa_exit'] = list(z_gwa_out)
fa.tree['x_camera_entrance'] = list(x_camera_entrance)
fa.tree['y_camera_entrance'] = list(y_camera_entrance)
fa.tree['x_fpa'] = list(x_fpa)
fa.tree['y_fpa'] = list(y_fpa)
fa.tree['x_sca_nrs1'] = list(x_sca_nrs1)
fa.tree['y_sca_nrs1'] = list(y_sca_nrs1)
fa.tree['x_sca_nrs2'] = list(x_sca_nrs2)
fa.tree['y_sca_nrs2'] = list(y_sca_nrs2)
fa.tree['x_oteip'] = list(x_oteip)
fa.tree['y_oteip'] = list(y_oteip)
fa.tree['v2'] = list(v2)
fa.tree['v3'] = list(v3)
fa.write_to("fixed_slits_functional.asdf", all_array_storage="internal")

## Computing the GWA correction

In [23]:
GWA_XTIL = 0.3316612243652344
GWA_YTIL = 0.1260581910610199

In [24]:
disp = datamodels.DisperserModel(refs['disperser'])
print('grating: ', disp.meta.instrument.grating)

grating:  G395H


In [25]:
theta_x = disp.theta_x
theta_y = disp.theta_y
tilt_y = disp.tilt_y

print('theta_x: ', theta_x)
print('theta_y: ', theta_y)
print('tilt_y: ', tilt_y)

theta_x:  0.023015465167583335
theta_y:  -0.03089246914388889
tilt_y:  -8.8


In [26]:
# Print C1, C2, zeroreadings
print('coefficients, tilt_y: ', disp._instance['gwa_tilty'])

coefficients, tilt_y:  {'temperatures': [37.17], 'unit': 'arcsec', 'tilt_model': <Polynomial1D(1, c0=0.0, c1=0.0)>, 'zeroreadings': [0.1448970586]}


In [27]:
print('coefficients, tilt_x: ', disp._instance['gwa_tiltx'])

coefficients, tilt_x:  {'temperatures': [37.17], 'unit': 'arcsec', 'tilt_model': <Polynomial1D(1, c0=2649.677, c1=-8196.032)>, 'zeroreadings': [0.323275774717]}


**Compute the correction for GWA_TILTY**

In [28]:
# φy_exposure = C1 × GWA_XTILexposure + C2
-8196.032*GWA_XTIL + 2649.677

-68.62900805664003

In [29]:
# φy_calibrator = C1 × Zeroreadings + C2 
-8196.032* 0.323275774717 + 2649.677

0.0984055946778426

In [30]:
# Δθcorrection = 0.5(φexposure − φcalibrator)
correction = 0.5 * (-68.62900805664003 - 0.0984055946778426) / 3600 # in deg
print('correction: ', correction)

correction:  -0.009545474118238594


In [31]:
corrected_theta_y = theta_y -0.009545474118238594
print('corrected_theta_y: ', corrected_theta_y)

corrected_theta_y:  -0.04043794326212748
